In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Pretpostavljeni DataFrame s ocjenama korisnika za filmove
ratings_data = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'rating': [5, 4, 4, 3, 5, 2, 3, 4]
})

# Učitavanje DataFrame-a sa žanrovima filmova
movies_data = pd.DataFrame({
    'movieId': [101, 102, 103],
    'genres': ['Action|Adventure|Sci-Fi', 'Comedy|Drama', 'Action|Drama|Thriller']
})

# Spajanje podataka o ocjenama i žanrovima
merged_data = pd.merge(ratings_data, movies_data, on='movieId')

# Funkcija za razdvajanje žanrova na jedinstvene tekstualne vrijednosti
def split_genres(data):
    unique_genres = set()
    for genres in data['genres']:
        unique_genres.update(genres.split('|'))
    return list(unique_genres)

# Razdvajanje žanrova na jedinstvene tekstualne vrijednosti
unique_genres = split_genres(movies_data)
print(merged_data)

# Računanje sličnosti između filmova pomoću kosinusne sličnosti
movie_similarity = cosine_similarity(merged_data[unique_genres])

# Funkcija za predviđanje preferiranih žanrova korisnika
def predict_user_genre_preferences(movie_similarity, user_id):
    # Izdvajanje ocjena za korisnika
    user_ratings = merged_data[merged_data['userId'] == user_id]

    # Računanje ponderiranih sličnosti između filmova i preferiranih žanrova korisnika
    weighted_similarity = np.dot(user_ratings[['Action', 'Adventure', 'Sci-Fi', 'Comedy', 'Drama', 'Thriller']], movie_similarity.T) / np.sum(np.abs(user_ratings[['Action', 'Adventure', 'Sci-Fi', 'Comedy', 'Drama', 'Thriller']]))

    # Normalizacija predviđenih ocjena u raspon od 0 do 1
    normalized_predictions = (weighted_similarity - np.min(weighted_similarity)) / (np.max(weighted_similarity) - np.min(weighted_similarity))

    # Mapiranje normaliziranih predviđenih ocjena na stvarne žanrove
    predicted_genres = [unique_genres[i] for i, value in enumerate(normalized_predictions) if value > 0]

    return predicted_genres

# Primjer generiranja preporuka za određenog korisnika (u ovom slučaju, korisnik s ID-om 1)
selected_user_id = 1
predicted_genres = predict_user_genre_preferences(movie_similarity, selected_user_id)

# Prikaz rezultata
print(f"Predicted genres for User ID {selected_user_id}:")
print(predicted_genres)


In [ ]:
import pandas as pd

# Primjer DataFrame-a sa žanrovima filmova
movies_data = pd.DataFrame({
    'movieId': [101, 102, 103],
    'genres': ['Action|Adventure|Sci-Fi', 'Comedy|Drama', 'Action|Drama|Thriller']
})

# Funkcija za razdvajanje žanrova na jedinstvene tekstualne vrijednosti
def split_genres(data):
    unique_genres = set()
    for genres in data['genres']:
        unique_genres.update(genres.split('|'))
    return list(unique_genres)

# Razdvajanje žanrova na jedinstvene tekstualne vrijednosti
unique_genres = split_genres(movies_data)

# Dodavanje stupaca za svaki jedinstveni žanr
for genre in unique_genres:
    movies_data[genre] = movies_data['genres'].apply(lambda x: 1 if genre in x else 0)

# Uklanjanje izvorne kolone 'genres'
movies_data.drop(columns=['genres'], inplace=True)

# Prikaz rezultata
print(movies_data)

# Pretpostavljeni DataFrame s ocjenama korisnika za filmove
user_ratings  = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'rating': [5, 4, 4, 3, 5, 2, 3, 4]
})

# Spajanje podataka o ocjenama i žanrovima
merged_data = pd.merge(ratings_data, movies_data, on='movieId')
user_movie_matrix = pd.pivot_table(user_ratings, values='rating', index='userId', columns='movieId', fill_value=0)
# Računanje sličnosti između filmova pomoću kosinusne sličnosti
# Računanje sličnosti između filmova pomoću kosinusne sličnosti
movie_similarity = cosine_similarity(movies_data[['Action', 'Adventure', 'Sci-Fi', 'Comedy', 'Drama', 'Thriller']])

# Funkcija za predviđanje preferiranih žanrova korisnika
def predict_user_genre_preferences(movie_similarity, user_ratings):
    weighted_similarity = np.dot(user_ratings, movie_similarity.T) / np.sum(np.abs(user_ratings))
    normalized_predictions = (weighted_similarity - np.min(weighted_similarity)) / (np.max(weighted_similarity) - np.min(weighted_similarity))
    return normalized_predictions

# Primjer generiranja preporuka za određenog korisnika (u ovom slučaju, korisnik s ID-om 1)
selected_user_id = 1

# Izdvajanje ocjena za korisnika
user_ratings = user_movie_matrix.loc[selected_user_id]

# Predviđanje preferiranih žanrova korisnika
predicted_preferences = predict_user_genre_preferences(movie_similarity, user_ratings)

# Mapiranje normaliziranih predviđenih ocjena na stvarne žanrove
predicted_genres = [movies_data.columns[i] for i, value in enumerate(predicted_preferences) if value > 0]

# Prikaz rezultata
print(f"Predicted genres for User ID {selected_user_id}:")
print(predicted_genres)